In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lg

In [2]:
# Reading the Dataset
dataset = pd.read_csv("Preprocessed_project_risk.csv")
dataset

,Team_Size,Project_Budget_USD,Estimated_Timeline_Months,Complexity_Score,Stakeholder_Count,Past_Similar_Projects,External_Dependencies_Count,Change_Request_Frequency,Team_Turnover_Rate,Vendor_Reliability_Score,...,Tech_Environment_Stability,Contract_Type,Resource_Contention_Level,Industry_Volatility,Client_Experience_Level,Change_Control_Maturity,Risk_Management_Maturity,Team_Colocation,Documentation_Quality,Risk_Level
0,32.0,1526276.550,32.0,9.70,16.0,3.0,3.0,1.05,0.16,0.84,...,0,3,0,0,0,2,2,0,2,High
1,2.0,390790.150,9.0,2.72,9.0,0.0,2.0,2.61,0.42,0.79,...,0,0,1,3,1,1,3,1,3,Low
2,2.0,246674.760,6.0,2.04,7.0,1.0,0.0,0.83,0.55,0.89,...,0,0,0,3,2,0,0,2,2,Medium
3,12.0,1427830.630,17.0,7.54,16.0,0.0,5.0,2.42,0.33,0.84,...,2,2,0,0,3,3,2,2,0,High
4,24.0,1696746.640,24.0,6.68,17.0,0.0,2.0,0.16,0.36,0.86,...,0,0,0,2,1,2,0,3,0,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,9.0,731548.970,14.0,5.40,9.0,4.0,3.0,1.09,0.25,0.94,...,1,1,2,2,1,0,0,2,0,High
3996,9.0,492981.400,9.0,2.69,8.0,5.0,3.0,2.11,0.06,0.71,...,0,1,2,3,1,0,2,2,0,Medium
3997,12.0,1097040.650,24.0,8.82,10.0,2.0,4.0,0.52,0.23,0.95,...,1,3,1,2,2,0,2,1,0,High
3998,26.0,2650876.115,28.0,10.00,23.0,1.0,6.0,1.19,0.30,0.50,...,0,1,2,1,3,2,3,1,2,High


In [3]:
# Separate target first
dependent = dataset["Risk_Level"]
independent = dataset.drop(columns=["Risk_Level"])

# Apply get_dummies only on independent
X_encoded = pd.get_dummies(independent, drop_first=True)
X_encoded=X_encoded.replace({True:1,False:0})
#from sklearn.preprocessing import LabelEncoder

#le = LabelEncoder()
#Y_encoded = le.fit_transform(dependent)
print(X_encoded.shape)
print(dependent.shape)

(4000, 50)
(4000,)


In [4]:
X_encoded

,Team_Size,Project_Budget_USD,Estimated_Timeline_Months,Complexity_Score,Stakeholder_Count,Past_Similar_Projects,External_Dependencies_Count,Change_Request_Frequency,Team_Turnover_Rate,Vendor_Reliability_Score,...,Key_Stakeholder_Availability,Tech_Environment_Stability,Contract_Type,Resource_Contention_Level,Industry_Volatility,Client_Experience_Level,Change_Control_Maturity,Risk_Management_Maturity,Team_Colocation,Documentation_Quality
0,32.0,1526276.550,32.0,9.70,16.0,3.0,3.0,1.05,0.16,0.84,...,2,0,3,0,0,0,2,2,0,2
1,2.0,390790.150,9.0,2.72,9.0,0.0,2.0,2.61,0.42,0.79,...,0,0,0,1,3,1,1,3,1,3
2,2.0,246674.760,6.0,2.04,7.0,1.0,0.0,0.83,0.55,0.89,...,3,0,0,0,3,2,0,0,2,2
3,12.0,1427830.630,17.0,7.54,16.0,0.0,5.0,2.42,0.33,0.84,...,1,2,2,0,0,3,3,2,2,0
4,24.0,1696746.640,24.0,6.68,17.0,0.0,2.0,0.16,0.36,0.86,...,3,0,0,0,2,1,2,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,9.0,731548.970,14.0,5.40,9.0,4.0,3.0,1.09,0.25,0.94,...,3,1,1,2,2,1,0,0,2,0
3996,9.0,492981.400,9.0,2.69,8.0,5.0,3.0,2.11,0.06,0.71,...,2,0,1,2,3,1,0,2,2,0
3997,12.0,1097040.650,24.0,8.82,10.0,2.0,4.0,0.52,0.23,0.95,...,3,1,3,1,2,2,0,2,1,0
3998,26.0,2650876.115,28.0,10.00,23.0,1.0,6.0,1.19,0.30,0.50,...,3,0,1,2,1,3,2,3,1,2


In [5]:
dependent

0         High
1          Low
2       Medium
3         High
4         High
         ...  
3995      High
3996    Medium
3997      High
3998      High
3999    Medium
Name: Risk_Level, Length: 4000, dtype: object

In [6]:
corr_matrix=X_encoded.corr()
corr_matrix

,Team_Size,Project_Budget_USD,Estimated_Timeline_Months,Complexity_Score,Stakeholder_Count,Past_Similar_Projects,External_Dependencies_Count,Change_Request_Frequency,Team_Turnover_Rate,Vendor_Reliability_Score,...,Key_Stakeholder_Availability,Tech_Environment_Stability,Contract_Type,Resource_Contention_Level,Industry_Volatility,Client_Experience_Level,Change_Control_Maturity,Risk_Management_Maturity,Team_Colocation,Documentation_Quality
Team_Size,1.000000,0.885798,0.864391,0.478861,0.785608,-0.006827,0.334217,0.018489,0.014237,-0.018919,...,-0.017918,-0.384641,0.007465,-0.091706,-0.191828,0.256410,-0.019352,-0.043273,-0.002134,-0.005299
Project_Budget_USD,0.885798,1.000000,0.864842,0.587870,0.886197,-0.002166,0.409297,0.012245,0.023810,-0.022124,...,-0.019448,-0.312069,-0.001072,-0.078846,-0.239588,0.405942,-0.011405,-0.040958,0.013866,-0.004091
Estimated_Timeline_Months,0.864391,0.864842,1.000000,0.792578,0.770566,-0.001003,0.553153,0.009925,0.034673,-0.017852,...,-0.016634,-0.268519,-0.002919,-0.082149,-0.399174,0.179607,-0.002150,-0.035316,0.000454,-0.006636
Complexity_Score,0.478861,0.587870,0.792578,1.000000,0.524190,0.009751,0.704350,-0.005723,0.028492,-0.012143,...,-0.001891,-0.037675,-0.016826,-0.032094,-0.550444,0.009352,0.017749,-0.028492,0.017547,-0.017939
Stakeholder_Count,0.785608,0.886197,0.770566,0.524190,1.000000,-0.009377,0.368666,0.017496,0.008317,-0.020938,...,-0.027536,-0.278372,0.012195,-0.063565,-0.209270,0.362912,-0.008534,-0.037213,0.013942,-0.013208
Past_Similar_Projects,-0.006827,-0.002166,-0.001003,0.009751,-0.009377,1.000000,-0.008743,0.004153,-0.317310,0.001897,...,0.019481,0.017176,-0.001965,-0.012671,0.003843,0.015380,0.042664,0.014756,0.000762,0.014705
External_Dependencies_Count,0.334217,0.409297,0.553153,0.704350,0.368666,-0.008743,1.000000,0.000334,0.038844,-0.026781,...,-0.003612,-0.020087,-0.014174,-0.010399,-0.394763,0.006156,0.017244,-0.025273,0.030017,0.012864
Change_Request_Frequency,0.018489,0.012245,0.009925,-0.005723,0.017496,0.004153,0.000334,1.000000,0.012088,-0.020419,...,0.022677,0.012039,-0.005754,0.021474,0.009978,0.008207,-0.000215,0.008340,-0.009248,0.005223
Team_Turnover_Rate,0.014237,0.023810,0.034673,0.028492,0.008317,-0.317310,0.038844,0.012088,1.000000,0.005883,...,0.032155,0.019552,-0.005924,0.011260,-0.033845,-0.002122,-0.012287,-0.017909,0.011277,-0.022773
Vendor_Reliability_Score,-0.018919,-0.022124,-0.017852,-0.012143,-0.020938,0.001897,-0.026781,-0.020419,0.005883,1.000000,...,-0.033128,0.014964,-0.004981,-0.005046,-0.009723,-0.011571,-0.005025,-0.005030,-0.002246,-0.014444


In [7]:
from lightgbm  import LGBMClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'num_leaves': [31, 60, 100],
                 'learning_rate': [0.01, 0.1],
                'feature_fraction': [0.7, 0.9]
}
grid = GridSearchCV(LGBMClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 

In [8]:
from sklearn.feature_selection import VarianceThreshold
var_selector = VarianceThreshold(threshold=0.01)
X_var = var_selector.fit_transform(X_encoded)

X_var_columns = X_encoded.columns[var_selector.get_support()]

from sklearn.feature_selection import RFE

rfe = RFE(estimator=grid,n_features_to_select=10,  # choose number of features    
          step=1)
X_rfe = rfe.fit_transform(X_var, dependent)
selected_features = X_var_columns.columns[rfe.get_support()]
X_selected = X_encoded[selected_features]
X_selected

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2570
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 48
[LightGBM] [Info] Start training from score -1.658103
[LightGBM] [Info] Start training from score -1.350927
[LightGBM] [Info] Start training from score -1.601966
[LightGBM] [Info] Start training from score -1.052683


ValueError: when `importance_getter=='auto'`, the underlying estimator GridSearchCV should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.

In [ ]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, dependent, test_size = 1/3, random_state =42)
X_train, X_test, Y_train, Y_test

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
 # fitting the model for grid search 
grid.fit(X_train, Y_train) 

In [ ]:
# print best parameter after tuning 
print(grid.best_params_) 
result=grid.cv_results_
print(result)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)

In [ ]:
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

In [ ]:
print("The confusion Matrix:\n",cm)

In [ ]:
print("The report:\n",clf_report)

In [ ]:
from sklearn.metrics import roc_auc_score

y_prob = grid.predict_proba(X_test)

roc_auc = roc_auc_score(
    Y_test,
    y_prob,
    multi_class="ovo",   # or "ovr"
    average="weighted"
)

print("ROC AUC:", roc_auc)



In [ ]:
best_model = grid.best_estimator_
importance_of_each_feature=best_model.feature_importances_
importance_of_each_feature

In [ ]:
X_selected.columns